In [1]:
ClientId = dbutils.secrets.get(scope = "Lambda", key = "ClientId")
ClientSecret = dbutils.secrets.get(scope = "Lambda", key = "ClientSecret")
TenantName = dbutils.secrets.get(scope = "Lambda", key = "TenantName")

In [2]:
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", ClientId)
spark.conf.set("dfs.adls.oauth2.credential", ClientSecret)
spark.conf.set("dfs.adls.oauth2.refresh.url", "https://login.microsoftonline.com/"+TenantName+"/oauth2/token")

In [3]:
## Load Data Frame ##
batchDf = sqlContext.read.format("com.databricks.spark.csv").option("header", "false").option("inferSchema", "true").load("adl://sawlake.azuredatalakestore.net/BASE/SalesTotals/Sales.csv")

batchDf = batchDf.withColumnRenamed("_c0", "ProductName").withColumnRenamed("_c1", "SalesTotal")

## Load Data Frame ##
speedDf = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema","true").load("adl://sawlake.azuredatalakestore.net/RAW/Corporate/SalesFeed/2018/05/30/10_-11419250_a1d4fabbfec64639bcd3fea9288c1f7a.csv")


batchDf.createOrReplaceTempView("batch")
speedDf.createOrReplaceTempView("speed")

In [4]:
%sql

select sum(Sales) TotalSales
from (
select sum(SalesTotal) as Sales from batch
union
select sum(salesamount) as Sales from speed
  )

TotalSales
8.126433431210072E7


In [5]:

%sql

select ProductName, sum(Sales) TotalSales
from (
select ProductName, sum(SalesTotal) as Sales from batch group by ProductName
union
select productname, sum(salesamount) as Sales from speed group by productname
  )
group by ProductName

ProductName,TotalSales
"Mountain-200 Black, 42",3788208.8436
"Touring-1000 Yellow, 46",1161042.09
"Touring-1000 Blue, 54",1046606.73
"Short-Sleeve Classic Jersey, S",58579.149999999994
"Women's Mountain Shorts, S",62291.09999999999
"Long-Sleeve Logo Jersey, L",61337.729999999996
"Mountain-400-W Silver, 42",268552.01
"Racing Socks, M",7497.66
"Mountain-100 Silver, 42",356998.94999999995
"Mountain-200 Silver, 42",3604767.2987999995
